In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Initializing/Import Data + Augmentation

In [2]:
csv_file = "dataset/data_labels_mainData.csv"
mainData_df = pd.read_csv(csv_file)

def load_data(dataframe, img_folder):
    data = []
    labels = []
    
    for index, row in dataframe.iterrows():
        img_name = row['ImageName']
        img_path = os.path.join(img_folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        data.append(img)
        labels.append(row['isCancerous'])
    
    return np.array(data), np.array(labels)

img_folder = "dataset/patch_images/"
data, labels = load_data(mainData_df, img_folder)




In [3]:
#changing isCancerous column to bool
mainData_df['isCancerous'] = mainData_df['isCancerous'].astype('bool')
# Normalize the image data
data = data.astype('float32') / 255.0

In [4]:
#image generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

# Apply data augmentation
augmented_data = []
augmented_labels = []

for img, label in zip(data, labels):
    img = img.reshape((1, *img.shape))
    for _ in range(3):  # Number of times to augment each image
        augmented_img = datagen.flow(img, batch_size=1)[0].reshape(img.shape[1:])
        augmented_data.append(augmented_img)
        augmented_labels.append(label)

augmented_data = np.array(augmented_data)
augmented_labels = np.array(augmented_labels)



## Model 1.1: Convolutional  Neural Network

In [5]:
#Building Neural Network Layers
cancerModel = Sequential()

cancerModel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(27, 27, 3)))
cancerModel.add(MaxPooling2D((2, 2)))

cancerModel.add(Conv2D(64, (3, 3), activation='relu'))
cancerModel.add(MaxPooling2D((2, 2)))

cancerModel.add(Conv2D(128, (3, 3), activation='relu'))
cancerModel.add(MaxPooling2D((2, 2)))

cancerModel.add(Flatten())
cancerModel.add(Dense(128, activation='relu'))
cancerModel.add(Dropout(0.5))
cancerModel.add(Dense(1, activation='softmax'))

In [6]:
# #Train test val split
# X_cnn_train, X_cnn_test, y_cnn_train, y_cnn_test = train_test_split(augmented_data, augmented_labels, test_size=0.3, random_state=2)

# # Compile the model
# cancerModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# history = cancerModel.fit(X_cnn_train, y_cnn_train,
#                         batch_size=32,
#                         epochs=50,
#                         validation_split=0.2,
#                         callbacks=[early_stopping])

# # Make predictions on the test set
# y_pred_cnn = np.argmax(cancerModel.predict(X_cnn_test), axis=-1)
# test_loss, test_accuracy = cancerModel.evaluate(X_cnn_test, y_cnn_test)

# # Calculate the accuracy, confusion matrix, and classification report
# test_loss, test_accuracy = cancerModel.evaluate(X_cnn_test, y_cnn_test)
# cnn_conf_mat = confusion_matrix(y_cnn_test, y_pred_cnn)
# cnn_class_report = classification_report(y_cnn_test, y_pred_cnn)

# print(f"Accuracy: {test_accuracy:.4f}")
# print(f"Loss: {test_loss:.4f}")
# print("\nConfusion Matrix:\n", cnn_conf_mat)
# print("\nClassification Report:\n", cnn_class_report)

Epoch 1/50
520/520 [==============================] - 5s 9ms/step - loss: 0.4505 - accuracy: 0.4133 - val_loss: 0.3128 - val_accuracy: 0.4138
Epoch 2/50
520/520 [==============================] - 4s 8ms/step - loss: 0.3329 - accuracy: 0.4133 - val_loss: 0.2982 - val_accuracy: 0.4138
Epoch 3/50
520/520 [==============================] - 4s 8ms/step - loss: 0.3106 - accuracy: 0.4133 - val_loss: 0.3082 - val_accuracy: 0.4138
Epoch 4/50
520/520 [==============================] - 4s 8ms/step - loss: 0.2953 - accuracy: 0.4133 - val_loss: 0.3090 - val_accuracy: 0.4138
Epoch 5/50
520/520 [==============================] - 4s 8ms/step - loss: 0.2849 - accuracy: 0.4133 - val_loss: 0.2615 - val_accuracy: 0.4138
Epoch 6/50
520/520 [==============================] - 4s 8ms/step - loss: 0.2769 - accuracy: 0.4133 - val_loss: 0.2932 - val_accuracy: 0.4138
Epoch 7/50
520/520 [==============================] - 4s 8ms/step - loss: 0.2671 - accuracy: 0.4133 - val_loss: 0.2633 - val_accuracy: 0.4138
Epoch 

C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model 1.2: Parameter Change: Activation Function + Learning Rate

In [7]:

#Building Neural Network Layers
cancersigModel = Sequential()

cancersigModel.add(Conv2D(32, (3, 3), activation='sigmoid', input_shape=(27, 27, 3)))
cancersigModel.add(MaxPooling2D((2, 2)))

cancersigModel.add(Conv2D(64, (3, 3), activation='sigmoid'))
cancersigModel.add(MaxPooling2D((2, 2)))

cancersigModel.add(Conv2D(128, (3, 3), activation='sigmoid'))
cancersigModel.add(MaxPooling2D((2, 2)))

cancersigModel.add(Flatten())
cancersigModel.add(Dense(128, activation='sigmoid'))
cancersigModel.add(Dropout(0.5))
cancersigModel.add(Dense(1, activation='sigmoid'))

In [8]:
# #Train test val split
# X_sig_train, X_sig_test, y_sig_train, y_sig_test = train_test_split(augmented_data, augmented_labels, test_size=0.3, random_state=2)

# # Compile the model
# cancersigModel.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# history = cancersigModel.fit(X_sig_train, y_sig_train,
#                         batch_size=32,
#                         epochs=50,
#                         validation_split=0.2,
#                         callbacks=[early_stopping])

# # Make predictions on the test set
# y_pred_sig = np.argmax(cancersigModel.predict(X_sig_test), axis=-1)

# # Calculate the accuracy, confusion matrix, and classification report
# test_loss, test_accuracy = cancersigModel.evaluate(X_sig_test, y_sig_test)
# sig_conf_mat = confusion_matrix(y_sig_test, y_pred_sig)
# sig_class_report = classification_report(y_sig_test, y_pred_sig)
# print(f"Accuracy: {test_accuracy:.4f}")
# print(f"Loss: {test_loss:.4f}")
# print("\nConfusion Matrix:\n", sig_conf_mat)
# print("\nClassification Report:\n", sig_class_report)

Epoch 1/50
520/520 [==============================] - 5s 9ms/step - loss: 0.6926 - accuracy: 0.5666 - val_loss: 0.6782 - val_accuracy: 0.5862
Epoch 2/50
520/520 [==============================] - 4s 8ms/step - loss: 0.6802 - accuracy: 0.5861 - val_loss: 0.6782 - val_accuracy: 0.5862
Epoch 3/50
520/520 [==============================] - 4s 8ms/step - loss: 0.6059 - accuracy: 0.6654 - val_loss: 0.5467 - val_accuracy: 0.7368
Epoch 4/50
520/520 [==============================] - 4s 8ms/step - loss: 0.4063 - accuracy: 0.8182 - val_loss: 0.3496 - val_accuracy: 0.8542
Epoch 5/50
520/520 [==============================] - 4s 8ms/step - loss: 0.3376 - accuracy: 0.8562 - val_loss: 0.3876 - val_accuracy: 0.8229
Epoch 6/50
520/520 [==============================] - 4s 9ms/step - loss: 0.3149 - accuracy: 0.8673 - val_loss: 0.2904 - val_accuracy: 0.8780
Epoch 7/50
520/520 [==============================] - 4s 8ms/step - loss: 0.3056 - accuracy: 0.8711 - val_loss: 0.3059 - val_accuracy: 0.8749
Epoch 

C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model 2: Random Forest

## HOG

In [9]:
#Train test val split
X_rfc_train, X_rfc_test, y_rfc_train, y_rfc_test = train_test_split(augmented_data, augmented_labels, test_size=0.3, random_state=2)



In [10]:
# def extract_hog_features(images):
#     features = []
#     for img in images:
#         hog_feature, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
#                     cells_per_block=(1, 1), visualize=True)
#         features.append(hog_feature)
#     return np.array(features)

# X_train_hog = extract_hog_features(X_rfc_train)
# X_test_hog = extract_hog_features(X_rfc_test)

## Random Forest Classifier

In [11]:
rfc_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rfc_classifier.fit(X_train_hog, y_rfc_train)

RandomForestClassifier(random_state=42)

## Determining the best hyperparameters

In [12]:
# param_grid = {
#     'n_estimators': [100, 200, 300],  
#     'max_depth': [None, 5, 10],      
#     'max_features': ['sqrt', 'log2'] 
# }

# gridsearch = GridSearchCV(estimator=rfc_classifier, param_grid=param_grid, cv=5)
# gridsearch.fit(X_train_hog, y_rfc_train)
# params = gridsearch.best_params_
# print("Hyperparameters:", params)

Hyperparameters: {'max_depth': 5, 'max_features': 'log2', 'n_estimators': 300}


### Training Random Forest Classifier

In [13]:
# # recreating model with new parameters
# reparam_rfc_classifier = RandomForestClassifier(**params)
# reparam_rfc_classifier.fit(X_train_hog, y_rfc_train)

# # Make predictions on the test data
# rfc_predictions = reparam_rfc_classifier.predict(X_test_hog)


### Evaluation

In [14]:

# # Evaluate the accuracy of the model
# rfc_accuracy = accuracy_score(y_rfc_test, rfc_predictions)
# rfc_conf_mat = confusion_matrix(y_rfc_test, rfc_predictions)
# rfc_class_report = classification_report(y_rfc_test, rfc_predictions)
# print(f"Accuracy: {rfc_accuracy:.4f}")
# print("\nConfusion Matrix:\n", rfc_conf_mat)
# print("\nClassification Report:\n", rfc_class_report)

Accuracy: 0.5905

Confusion Matrix:
 [[5260    0]
 [3647    0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.59      1.00      0.74      5260
           1       0.00      0.00      0.00      3647

    accuracy                           0.59      8907
   macro avg       0.30      0.50      0.37      8907
weighted avg       0.35      0.59      0.44      8907



C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Oscar\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model 3: CNN + Random Forest

### From CNN sig in Model 1.2

**NOTE: Model 1.2 code MUST be run before using Model 3**

In [15]:
target_layer = tf.keras.Model(inputs=cancersigModel.input, outputs=cancersigModel.layers[-2].output)

def extract_sig_features(images):
    features = target_layer.predict(images)
    return features

train_sig_features = extract_sig_features(X_sig_train)
train_sig_features = train_sig_features.reshape(train_sig_features.shape[0], -1)


650/650 [==============================] - 2s 3ms/step


### Training Random Forest Classifier

In [16]:
rfc_classifier = RandomForestClassifier()

rfc_classifier.fit(train_sig_features, y_sig_train)

RandomForestClassifier()

### Evaluation

In [17]:
test_sig_features = extract_sig_features(X_sig_test)
test_sig_features = test_sig_features.reshape(test_sig_features.shape[0], -1)
rfsig_predictions = rfc_classifier.predict(test_sig_features)

279/279 [==============================] - 1s 3ms/step


In [18]:
rfsig_accuracy = accuracy_score(y_sig_test, rfsig_predictions)
rfsig_conf_mat = confusion_matrix(y_sig_test, rfsig_predictions)
rfsig_class_report = classification_report(y_sig_test, rfsig_predictions)

print(f"Accuracy: {rfsig_accuracy:.4f}")
print("\nConfusion Matrix:\n", rfsig_conf_mat)
print("\nClassification Report:\n", rfsig_class_report)

Accuracy: 0.8923

Confusion Matrix:
 [[4818  442]
 [ 517 3130]]

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.92      0.91      5260
           1       0.88      0.86      0.87      3647

    accuracy                           0.89      8907
   macro avg       0.89      0.89      0.89      8907
weighted avg       0.89      0.89      0.89      8907

